In [1]:
import os
from tqdm import tqdm
# import feather
import pandas as pd
import numpy as np
from scipy import stats
import gc
from joblib import Parallel, delayed

In [2]:
def append_to_csv(batch, csv_file):
    props = dict(encoding='utf-8', index=False)
    if not os.path.exists(csv_file):
        batch.to_csv(csv_file, **props)
    else:
        batch.to_csv(csv_file, mode='a', header=False, **props)

def delete_file_if_exists(filename):
    if os.path.exists(filename):
        os.remove(filename)

In [3]:
fn = '../cache/bkup/train10_t_store{}.csv'.format(1)
df =pd.read_csv(fn)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# df.head(2000)

In [5]:
# df[df.onpromotion == True]

In [6]:
# fn = '../cache/bkup/train10_t_store{}.csv'.format(2)
# df2 =pd.read_csv(fn)

In [7]:
df[(df.onpromotion == True) & (df.store_item_nbr == '1_105737')]

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,item_family,item_class,perishable,city,...,dow,doy,dcount,h_type,h_desc,pd,wbe,wae,wfe,store_item_nbr


In [8]:
# len(set(df2.store_item_nbr))

In [9]:
# df2[df2.onpromotion == True]

In [10]:
# df2.h_type.value_counts()

In [11]:
# df2.h_desc.value_counts()

In [12]:
def days_from_next_promotion(store_num):
    fn = '../cache/bkup/train10_t_store{}.csv'.format(store_num)
    df =pd.read_csv(fn)
    df['dts'] = df.date.map(np.datetime64)
    df_min_date = np.min(df.dts)
    df_max_date = np.min(df.dts)
    max_days = ((df_max_date - df_min_date))/np.timedelta64(1, 'D')
    n = 0
    items = list(set(df.store_item_nbr))
    for item in items:
        n = 1
        df_it = df[(df.store_item_nbr == item)].reset_index(drop=True)
        
        df_it['pe'] = 0 # promotion event
        df_it['pe'][df_it.onpromotion == True] = 1
        promo_indices = list(np.transpose(*np.where(df_it.pe)))
        
        df_it['days_from_next_promo'] = max_days
        last_loc = 0
        for j in promo_indices:
            promo_date = df_it.loc[j, 'dts']
            while last_loc < j:
                curr_date = df_it.loc[last_loc, 'dts']
                if(curr_date == promo_date):
                    df_it.loc[last_loc, 'days_from_next_promo'] = 0
                    continue
                diff = int((promo_date - curr_date)/ np.timedelta64(1, 'D'))
                df_it.loc[last_loc, 'days_from_next_promo'] = diff
                last_loc += 1    
            if last_loc == j:
                df_it.loc[last_loc, 'days_from_next_promo'] = 0
                last_loc += 1
        fn_tmp = '../cache/train10_t_store{}_{}_next_promo_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)
        df1 = pd.DataFrame()
        df1['id'] = df_it['id'].values
        df1['days_from_next_promo'] = df_it['days_from_next_promo'].values
        df1.to_csv(fn_tmp, float_format='%.5f', index=False)
        del df1
    df2 = pd.DataFrame()
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_next_promo_tmp.csv'.format(store_num, item)
        df1 = pd.read_csv(fn_tmp)
        df2 = pd.concat([df2, df1], axis=0)
    df2 = df2.sort('id')
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]
    min_id = df.iloc[0]['id']
    df2 = df2[df2.id >= min_id]
    assert(len(df) == len(df2))
    # assert(df.id == df2.id)
    s = sum(df2.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df2.isnull().sum())
    fn_tmp = '../cache/train10_t_store{}_days_from_next_promo.csv'.format(store_num)
    delete_file_if_exists(fn_tmp)
    df2.to_csv(fn_tmp, float_format='%.5f', index=False)
    del df2
    del df
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_next_promo_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)

In [13]:
def days_from_prev_promotion(store_num):
    fn = '../cache/bkup/train10_t_store{}.csv'.format(store_num)
    df =pd.read_csv(fn)
    df['dts'] = df.date.map(np.datetime64)
    df_min_date = np.min(df.dts)
    df_max_date = np.min(df.dts)
    max_days = ((df_max_date - df_min_date))/np.timedelta64(1, 'D')
    n = 0
    items = list(set(df.store_item_nbr))
    for item in items:
        n = 1
        df_it = df[(df.store_item_nbr == item)].reset_index(drop=True)
        
        df_it['pe'] = 0 # promotion event
        df_it['pe'][df_it.onpromotion == True] = 1
        promo_indices = list(np.transpose(*np.where(df_it.pe)))
        
        df_it['days_from_prev_promo'] = max_days
        last_loc = len(df_it)-1
        for j in promo_indices:
            promo_date = df_it.loc[j, 'dts']
            while last_loc > j:
                curr_date = df_it.loc[last_loc, 'dts']
                if(curr_date == promo_date):
                    df_it.loc[last_loc, 'days_from_prev_promo'] = 0
                    continue
                diff = int((promo_date - curr_date)/ np.timedelta64(1, 'D'))
                df_it.loc[last_loc, 'days_from_prev_promo'] = abs(diff)
                last_loc -= 1    
            if last_loc == j:
                df_it.loc[last_loc, 'days_from_prev_promo'] = 0
                last_loc -= 1
        fn_tmp = '../cache/train10_t_store{}_{}_prev_promo_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)
        df1 = pd.DataFrame()
        df1['id'] = df_it['id'].values
        df1['days_from_prev_promo'] = df_it['days_from_prev_promo'].values
        df1.to_csv(fn_tmp, float_format='%.5f', index=False)
        del df1
    df2 = pd.DataFrame()
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_prev_promo_tmp.csv'.format(store_num, item)
        df1 = pd.read_csv(fn_tmp)
        df2 = pd.concat([df2, df1], axis=0)
    df2 = df2.sort('id')
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]
    min_id = df.iloc[0]['id']
    df2 = df2[df2.id >= min_id]
    assert(len(df) == len(df2))
    # assert(df.id == df2.id)
    s = sum(df2.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df2.isnull().sum())
    fn_tmp = '../cache/train10_t_store{}_days_from_prev_promo.csv'.format(store_num)
    delete_file_if_exists(fn_tmp)
    df2.to_csv(fn_tmp, float_format='%.5f', index=False)
    del df2
    del df
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_prev_promo_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)

In [14]:
def days_from_next_event(store_num):
    fn = '../cache/bkup/train10_t_store{}.csv'.format(store_num)
    df =pd.read_csv(fn)
    df['dts'] = df.date.map(np.datetime64)
    df_min_date = np.min(df.dts)
    df_max_date = np.min(df.dts)
    max_days = ((df_max_date - df_min_date))/np.timedelta64(1, 'D')
    n = 0
    items = list(set(df.store_item_nbr))
    for item in items:
        n = 1
        df_it = df[(df.store_item_nbr == item)].reset_index(drop=True)
        
        df_it['pe'] = 0 # promotion event
        df_it['pe'][df_it.h_type == 'Event'] = 1
        promo_indices = list(np.transpose(*np.where(df_it.pe)))
        
        df_it['days_from_next_event'] = max_days
        last_loc = 0
        for j in promo_indices:
            promo_date = df_it.loc[j, 'dts']
            while last_loc < j:
                curr_date = df_it.loc[last_loc, 'dts']
                if(curr_date == promo_date):
                    df_it.loc[last_loc, 'days_from_next_event'] = 0
                    continue
                diff = int((promo_date - curr_date)/ np.timedelta64(1, 'D'))
                df_it.loc[last_loc, 'days_from_next_event'] = diff
                last_loc += 1    
            if last_loc == j:
                df_it.loc[last_loc, 'days_from_next_event'] = 0
                last_loc += 1
        fn_tmp = '../cache/train10_t_store{}_{}_next_event_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)
        df1 = pd.DataFrame()
        df1['id'] = df_it['id'].values
        df1['days_from_next_event'] = df_it['days_from_next_event'].values
        df1.to_csv(fn_tmp, float_format='%.5f', index=False)
        del df1
    df2 = pd.DataFrame()
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_next_event_tmp.csv'.format(store_num, item)
        df1 = pd.read_csv(fn_tmp)
        df2 = pd.concat([df2, df1], axis=0)
    df2 = df2.sort('id')
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]
    min_id = df.iloc[0]['id']
    df2 = df2[df2.id >= min_id]
    assert(len(df) == len(df2))
    # assert(df.id == df2.id)
    s = sum(df2.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df2.isnull().sum())
    fn_tmp = '../cache/train10_t_store{}_days_from_next_event.csv'.format(store_num)
    delete_file_if_exists(fn_tmp)
    df2.to_csv(fn_tmp, float_format='%.5f', index=False)
    del df2
    del df
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_next_event_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)

In [15]:
def days_from_prev_event(store_num):
    fn = '../cache/bkup/train10_t_store{}.csv'.format(store_num)
    df =pd.read_csv(fn)
    df['dts'] = df.date.map(np.datetime64)
    df_min_date = np.min(df.dts)
    df_max_date = np.min(df.dts)
    max_days = ((df_max_date - df_min_date))/np.timedelta64(1, 'D')
    n = 0
    items = list(set(df.store_item_nbr))
    for item in items:
        n = 1
        df_it = df[(df.store_item_nbr == item)].reset_index(drop=True)
        
        df_it['pe'] = 0 # promotion event
        df_it['pe'][df_it.h_type == 'Event'] = 1
        promo_indices = list(np.transpose(*np.where(df_it.pe)))
        
        df_it['days_from_prev_event'] = max_days
        last_loc = len(df_it)-1
        for j in promo_indices:
            promo_date = df_it.loc[j, 'dts']
            while last_loc > j:
                curr_date = df_it.loc[last_loc, 'dts']
                if(curr_date == promo_date):
                    df_it.loc[last_loc, 'days_from_prev_event'] = 0
                    continue
                diff = int((promo_date - curr_date)/ np.timedelta64(1, 'D'))
                df_it.loc[last_loc, 'days_from_prev_event'] = abs(diff)
                last_loc -= 1    
            if last_loc == j:
                df_it.loc[last_loc, 'days_from_prev_event'] = 0
                last_loc -= 1
        fn_tmp = '../cache/train10_t_store{}_{}_prev_event_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)
        df1 = pd.DataFrame()
        df1['id'] = df_it['id'].values
        df1['days_from_prev_event'] = df_it['days_from_prev_event'].values
        df1.to_csv(fn_tmp, float_format='%.5f', index=False)
        del df1
    df2 = pd.DataFrame()
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_prev_event_tmp.csv'.format(store_num, item)
        df1 = pd.read_csv(fn_tmp)
        df2 = pd.concat([df2, df1], axis=0)
    df2 = df2.sort('id')
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]
    min_id = df.iloc[0]['id']
    df2 = df2[df2.id >= min_id]
    assert(len(df) == len(df2))
    # assert(df.id == df2.id)
    s = sum(df2.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df2.isnull().sum())
    fn_tmp = '../cache/train10_t_store{}_days_from_prev_event.csv'.format(store_num)
    delete_file_if_exists(fn_tmp)
    df2.to_csv(fn_tmp, float_format='%.5f', index=False)
    del df2
    del df
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_prev_event_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)

In [16]:
def days_from_next_holiday(store_num):
    fn = '../cache/bkup/train10_t_store{}.csv'.format(store_num)
    df =pd.read_csv(fn)
    df['dts'] = df.date.map(np.datetime64)
    df_min_date = np.min(df.dts)
    df_max_date = np.min(df.dts)
    max_days = ((df_max_date - df_min_date))/np.timedelta64(1, 'D')
    n = 0
    items = list(set(df.store_item_nbr))
    for item in items:
        n = 1
        df_it = df[(df.store_item_nbr == item)].reset_index(drop=True)
        
        df_it['pe'] = 0 # promotion event
        df_it['pe'][df_it.h_type == 'Holiday'] = 1
        promo_indices = list(np.transpose(*np.where(df_it.pe)))
        
        df_it['days_from_next_holiday'] = max_days
        last_loc = 0
        for j in promo_indices:
            promo_date = df_it.loc[j, 'dts']
            while last_loc < j:
                curr_date = df_it.loc[last_loc, 'dts']
                if(curr_date == promo_date):
                    df_it.loc[last_loc, 'days_from_next_holiday'] = 0
                    continue
                diff = int((promo_date - curr_date)/ np.timedelta64(1, 'D'))
                df_it.loc[last_loc, 'days_from_next_holiday'] = diff
                last_loc += 1    
            if last_loc == j:
                df_it.loc[last_loc, 'days_from_next_holiday'] = 0
                last_loc += 1
        fn_tmp = '../cache/train10_t_store{}_{}_next_holiday_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)
        df1 = pd.DataFrame()
        df1['id'] = df_it['id'].values
        df1['days_from_next_holiday'] = df_it['days_from_next_holiday'].values
        df1.to_csv(fn_tmp, float_format='%.5f', index=False)
        del df1
    df2 = pd.DataFrame()
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_next_holiday_tmp.csv'.format(store_num, item)
        df1 = pd.read_csv(fn_tmp)
        df2 = pd.concat([df2, df1], axis=0)
    df2 = df2.sort('id')
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]
    min_id = df.iloc[0]['id']
    df2 = df2[df2.id >= min_id]
    assert(len(df) == len(df2))
    # assert(df.id == df2.id)
    s = sum(df2.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df2.isnull().sum())
    fn_tmp = '../cache/train10_t_store{}_days_from_next_holiday.csv'.format(store_num)
    delete_file_if_exists(fn_tmp)
    df2.to_csv(fn_tmp, float_format='%.5f', index=False)
    del df2
    del df
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_next_holiday_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)

In [17]:
def days_from_prev_holiday(store_num):
    fn = '../cache/bkup/train10_t_store{}.csv'.format(store_num)
    df =pd.read_csv(fn)
    df['dts'] = df.date.map(np.datetime64)
    df_min_date = np.min(df.dts)
    df_max_date = np.min(df.dts)
    max_days = ((df_max_date - df_min_date))/np.timedelta64(1, 'D')
    n = 0
    items = list(set(df.store_item_nbr))
    for item in items:
        n = 1
        df_it = df[(df.store_item_nbr == item)].reset_index(drop=True)
        
        df_it['pe'] = 0 # promotion event
        df_it['pe'][df_it.h_type == 'Holiday'] = 1
        promo_indices = list(np.transpose(*np.where(df_it.pe)))
        
        df_it['days_from_prev_holiday'] = max_days
        last_loc = len(df_it)-1
        for j in promo_indices:
            promo_date = df_it.loc[j, 'dts']
            while last_loc > j:
                curr_date = df_it.loc[last_loc, 'dts']
                if(curr_date == promo_date):
                    df_it.loc[last_loc, 'days_from_prev_holiday'] = 0
                    continue
                diff = int((promo_date - curr_date)/ np.timedelta64(1, 'D'))
                df_it.loc[last_loc, 'days_from_prev_holiday'] = abs(diff)
                last_loc -= 1    
            if last_loc == j:
                df_it.loc[last_loc, 'days_from_prev_holiday'] = 0
                last_loc -= 1
        fn_tmp = '../cache/train10_t_store{}_{}_prev_holiday_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)
        df1 = pd.DataFrame()
        df1['id'] = df_it['id'].values
        df1['days_from_prev_holiday'] = df_it['days_from_prev_holiday'].values
        df1.to_csv(fn_tmp, float_format='%.5f', index=False)
        del df1
    df2 = pd.DataFrame()
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_prev_holiday_tmp.csv'.format(store_num, item)
        df1 = pd.read_csv(fn_tmp)
        df2 = pd.concat([df2, df1], axis=0)
    df2 = df2.sort('id')
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]
    min_id = df.iloc[0]['id']
    df2 = df2[df2.id >= min_id]
    assert(len(df) == len(df2))
    # assert(df.id == df2.id)
    s = sum(df2.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df2.isnull().sum())
    fn_tmp = '../cache/train10_t_store{}_days_from_prev_holiday.csv'.format(store_num)
    delete_file_if_exists(fn_tmp)
    df2.to_csv(fn_tmp, float_format='%.5f', index=False)
    del df2
    del df
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_prev_holiday_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)

In [18]:
def days_from_next_nw(store_num):
    fn = '../cache/bkup/train10_t_store{}.csv'.format(store_num)
    df =pd.read_csv(fn)
    df['dts'] = df.date.map(np.datetime64)
    df_min_date = np.min(df.dts)
    df_max_date = np.min(df.dts)
    max_days = ((df_max_date - df_min_date))/np.timedelta64(1, 'D')
    n = 0
    items = list(set(df.store_item_nbr))
    for item in items:
        n = 1
        df_it = df[(df.store_item_nbr == item)].reset_index(drop=True)
        
        df_it['pe'] = 0 # promotion event
        df_it['pe'][df_it.h_type != 'Work Day'] = 1
        promo_indices = list(np.transpose(*np.where(df_it.pe)))
        
        df_it['days_from_next_nw'] = max_days
        last_loc = 0
        for j in promo_indices:
            promo_date = df_it.loc[j, 'dts']
            while last_loc < j:
                curr_date = df_it.loc[last_loc, 'dts']
                if(curr_date == promo_date):
                    df_it.loc[last_loc, 'days_from_next_nw'] = 0
                    continue
                diff = int((promo_date - curr_date)/ np.timedelta64(1, 'D'))
                df_it.loc[last_loc, 'days_from_next_nw'] = diff
                last_loc += 1    
            if last_loc == j:
                df_it.loc[last_loc, 'days_from_next_nw'] = 0
                last_loc += 1
        fn_tmp = '../cache/train10_t_store{}_{}_next_nw_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)
        df1 = pd.DataFrame()
        df1['id'] = df_it['id'].values
        df1['days_from_next_nw'] = df_it['days_from_next_nw'].values
        df1.to_csv(fn_tmp, float_format='%.5f', index=False)
        del df1
    df2 = pd.DataFrame()
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_next_nw_tmp.csv'.format(store_num, item)
        df1 = pd.read_csv(fn_tmp)
        df2 = pd.concat([df2, df1], axis=0)
    df2 = df2.sort('id')
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]
    min_id = df.iloc[0]['id']
    df2 = df2[df2.id >= min_id]
    assert(len(df) == len(df2))
    # assert(df.id == df2.id)
    s = sum(df2.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df2.isnull().sum())
    fn_tmp = '../cache/train10_t_store{}_days_from_next_nw.csv'.format(store_num)
    delete_file_if_exists(fn_tmp)
    df2.to_csv(fn_tmp, float_format='%.5f', index=False)
    del df2
    del df
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_next_nw_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)

In [19]:
def days_from_prev_nw(store_num):
    fn = '../cache/bkup/train10_t_store{}.csv'.format(store_num)
    df =pd.read_csv(fn)
    df['dts'] = df.date.map(np.datetime64)
    df_min_date = np.min(df.dts)
    df_max_date = np.min(df.dts)
    max_days = ((df_max_date - df_min_date))/np.timedelta64(1, 'D')
    n = 0
    items = list(set(df.store_item_nbr))
    for item in items:
        n = 1
        df_it = df[(df.store_item_nbr == item)].reset_index(drop=True)
        
        df_it['pe'] = 0 # promotion event
        df_it['pe'][df_it.h_type != 'Work Day'] = 1
        promo_indices = list(np.transpose(*np.where(df_it.pe)))
        
        df_it['days_from_prev_nw'] = max_days
        last_loc = len(df_it)-1
        for j in promo_indices:
            promo_date = df_it.loc[j, 'dts']
            while last_loc > j:
                curr_date = df_it.loc[last_loc, 'dts']
                if(curr_date == promo_date):
                    df_it.loc[last_loc, 'days_from_prev_nw'] = 0
                    continue
                diff = int((promo_date - curr_date)/ np.timedelta64(1, 'D'))
                df_it.loc[last_loc, 'days_from_prev_nw'] = abs(diff)
                last_loc -= 1    
            if last_loc == j:
                df_it.loc[last_loc, 'days_from_prev_nw'] = 0
                last_loc -= 1
        fn_tmp = '../cache/train10_t_store{}_{}_prev_nw_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)
        df1 = pd.DataFrame()
        df1['id'] = df_it['id'].values
        df1['days_from_prev_nw'] = df_it['days_from_prev_nw'].values
        df1.to_csv(fn_tmp, float_format='%.5f', index=False)
        del df1
    df2 = pd.DataFrame()
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_prev_nw_tmp.csv'.format(store_num, item)
        df1 = pd.read_csv(fn_tmp)
        df2 = pd.concat([df2, df1], axis=0)
    df2 = df2.sort('id')
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]
    min_id = df.iloc[0]['id']
    df2 = df2[df2.id >= min_id]
    assert(len(df) == len(df2))
    # assert(df.id == df2.id)
    s = sum(df2.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df2.isnull().sum())
    fn_tmp = '../cache/train10_t_store{}_days_from_prev_nw.csv'.format(store_num)
    delete_file_if_exists(fn_tmp)
    df2.to_csv(fn_tmp, float_format='%.5f', index=False)
    del df2
    del df
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_prev_nw_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)

In [20]:
def days_from_next_nn(store_num):
    fn = '../cache/bkup/train10_t_store{}.csv'.format(store_num)
    df =pd.read_csv(fn)
    df['dts'] = df.date.map(np.datetime64)
    df_min_date = np.min(df.dts)
    df_max_date = np.min(df.dts)
    max_days = ((df_max_date - df_min_date))/np.timedelta64(1, 'D')
    n = 0
    items = list(set(df.store_item_nbr))
    for item in items:
        n = 1
        df_it = df[(df.store_item_nbr == item)].reset_index(drop=True)
        
        df_it['pe'] = 0 # promotion event
        df_it['pe'][df_it.h_desc != 'Normal'] = 1
        promo_indices = list(np.transpose(*np.where(df_it.pe)))
        
        df_it['days_from_next_nn'] = max_days
        last_loc = 0
        for j in promo_indices:
            promo_date = df_it.loc[j, 'dts']
            while last_loc < j:
                curr_date = df_it.loc[last_loc, 'dts']
                if(curr_date == promo_date):
                    df_it.loc[last_loc, 'days_from_next_nn'] = 0
                    continue
                diff = int((promo_date - curr_date)/ np.timedelta64(1, 'D'))
                df_it.loc[last_loc, 'days_from_next_nn'] = diff
                last_loc += 1    
            if last_loc == j:
                df_it.loc[last_loc, 'days_from_next_nn'] = 0
                last_loc += 1
        fn_tmp = '../cache/train10_t_store{}_{}_next_nn_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)
        df1 = pd.DataFrame()
        df1['id'] = df_it['id'].values
        df1['days_from_next_nn'] = df_it['days_from_next_nn'].values
        df1.to_csv(fn_tmp, float_format='%.5f', index=False)
        del df1
    df2 = pd.DataFrame()
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_next_nn_tmp.csv'.format(store_num, item)
        df1 = pd.read_csv(fn_tmp)
        df2 = pd.concat([df2, df1], axis=0)
    df2 = df2.sort('id')
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]
    min_id = df.iloc[0]['id']
    df2 = df2[df2.id >= min_id]
    assert(len(df) == len(df2))
    # assert(df.id == df2.id)
    s = sum(df2.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df2.isnull().sum())
    fn_tmp = '../cache/train10_t_store{}_days_from_next_nn.csv'.format(store_num)
    delete_file_if_exists(fn_tmp)
    df2.to_csv(fn_tmp, float_format='%.5f', index=False)
    del df2
    del df
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_next_nn_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)

In [21]:
def days_from_prev_nn(store_num):
    fn = '../cache/bkup/train10_t_store{}.csv'.format(store_num)
    df =pd.read_csv(fn)
    df['dts'] = df.date.map(np.datetime64)
    df_min_date = np.min(df.dts)
    df_max_date = np.min(df.dts)
    max_days = ((df_max_date - df_min_date))/np.timedelta64(1, 'D')
    n = 0
    items = list(set(df.store_item_nbr))
    for item in items:
        n = 1
        df_it = df[(df.store_item_nbr == item)].reset_index(drop=True)
        
        df_it['pe'] = 0 # promotion event
        df_it['pe'][df_it.h_desc != 'Normal'] = 1
        promo_indices = list(np.transpose(*np.where(df_it.pe)))
        
        df_it['days_from_prev_nn'] = max_days
        last_loc = len(df_it)-1
        for j in promo_indices:
            promo_date = df_it.loc[j, 'dts']
            while last_loc > j:
                curr_date = df_it.loc[last_loc, 'dts']
                if(curr_date == promo_date):
                    df_it.loc[last_loc, 'days_from_prev_nn'] = 0
                    continue
                diff = int((promo_date - curr_date)/ np.timedelta64(1, 'D'))
                df_it.loc[last_loc, 'days_from_prev_nn'] = abs(diff)
                last_loc -= 1    
            if last_loc == j:
                df_it.loc[last_loc, 'days_from_prev_nn'] = 0
                last_loc -= 1
        fn_tmp = '../cache/train10_t_store{}_{}_prev_nn_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)
        df1 = pd.DataFrame()
        df1['id'] = df_it['id'].values
        df1['days_from_prev_nn'] = df_it['days_from_prev_nn'].values
        df1.to_csv(fn_tmp, float_format='%.5f', index=False)
        del df1
    df2 = pd.DataFrame()
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_prev_nn_tmp.csv'.format(store_num, item)
        df1 = pd.read_csv(fn_tmp)
        df2 = pd.concat([df2, df1], axis=0)
    df2 = df2.sort('id')
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]
    min_id = df.iloc[0]['id']
    df2 = df2[df2.id >= min_id]
    assert(len(df) == len(df2))
    # assert(df.id == df2.id)
    s = sum(df2.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df2.isnull().sum())
    fn_tmp = '../cache/train10_t_store{}_days_from_prev_nn.csv'.format(store_num)
    delete_file_if_exists(fn_tmp)
    df2.to_csv(fn_tmp, float_format='%.5f', index=False)
    del df2
    del df
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_prev_nn_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)

In [22]:
# days_from_next_promotion(df2)

In [23]:
# for i in tqdm(range(1,55)):
#     days_from_next_promotion(i)
#     days_from_prev_promotion(i)
#     days_from_next_event(i)
#     days_from_prev_event(i)
#     days_from_next_holiday(i)
#     days_from_prev_holiday(i)
#     days_from_next_nw(i)
#     days_from_prev_nw(i)
#     days_from_next_nn(i)
#     days_from_prev_nn(i)

In [24]:
def days_from_prev_nw_test(store_num):
    fn = '../cache/bkup/test7_t_store{}.csv'.format(store_num)
    df =pd.read_csv(fn)
    df['dts'] = df.date.map(np.datetime64)
    df_min_date = np.min(df.dts)
    df_max_date = np.min(df.dts)
    max_days = ((df_max_date - df_min_date))/np.timedelta64(1, 'D')
    n = 0
    items = list(set(df.store_item_nbr))
    for item in items:
        n = 1
        df_it = df[(df.store_item_nbr == item)].reset_index(drop=True)
        
        df_it['pe'] = 0 # promotion event
        df_it['pe'][df_it.h_type != 'Work Day'] = 1
        promo_indices = list(np.transpose(*np.where(df_it.pe)))
        
        df_it['days_from_prev_nw'] = max_days
        last_loc = len(df_it)-1
        for j in promo_indices:
            promo_date = df_it.loc[j, 'dts']
            while last_loc > j:
                curr_date = df_it.loc[last_loc, 'dts']
                if(curr_date == promo_date):
                    df_it.loc[last_loc, 'days_from_prev_nw'] = 0
                    continue
                diff = int((promo_date - curr_date)/ np.timedelta64(1, 'D'))
                df_it.loc[last_loc, 'days_from_prev_nw'] = abs(diff)
                last_loc -= 1    
            if last_loc == j:
                df_it.loc[last_loc, 'days_from_prev_nw'] = 0
                last_loc -= 1
        fn_tmp = '../cache/test7_t_store{}_{}_prev_nw_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)
        df1 = pd.DataFrame()
        df1['id'] = df_it['id'].values
        df1['days_from_prev_nw'] = df_it['days_from_prev_nw'].values
        df1.to_csv(fn_tmp, float_format='%.5f', index=False)
        del df1
    df2 = pd.DataFrame()
    for item in items:
        fn_tmp = '../cache/test7_t_store{}_{}_prev_nw_tmp.csv'.format(store_num, item)
        df1 = pd.read_csv(fn_tmp)
        df2 = pd.concat([df2, df1], axis=0)
    df2 = df2.sort('id')
    df = df[(df.dts >= np.datetime64('2017-08-16')) & (df.dts <= np.datetime64('2017-08-31'))]
    min_id = df.iloc[0]['id']
    df2 = df2[df2.id >= min_id]
    assert(len(df) == len(df2))
    # assert(df.id == df2.id)
    s = sum(df2.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df2.isnull().sum())
    fn_tmp = '../cache/test7_t_store{}_days_from_prev_nw.csv'.format(store_num)
    delete_file_if_exists(fn_tmp)
    df2.to_csv(fn_tmp, float_format='%.5f', index=False)
    del df2
    del df
    for item in items:
        fn_tmp = '../cache/test7_t_store{}_{}_prev_nw_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)

In [25]:
for i in tqdm(range(1,55)):
#     days_from_next_promotion(i)
#     days_from_prev_promotion(i)
#     days_from_next_event(i)
#     days_from_prev_event(i)
#     days_from_next_holiday(i)
#     days_from_prev_holiday(i)
#     days_from_next_nw(i)
    days_from_prev_nw_test(i)
#     days_from_next_nn(i)
#     days_from_prev_nn(i)

  0%|          | 0/54 [00:00<?, ?it/s]/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:45: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


0


  4%|▎         | 2/54 [09:46<4:10:57, 289.56s/it]

0
0


  6%|▌         | 3/54 [14:55<4:10:58, 295.27s/it]

...

0
0


 98%|█████████▊| 53/54 [4:10:44<03:21, 201.38s/it]

0


100%|██████████| 54/54 [4:13:57<00:00, 198.95s/it]

0
